In [1]:
df = read.csv('EoE_TCR_190322_PoolF_EoE_TR1V_TCRalignsortsortMappingSummary.txt', sep = '\t', header = 1, stringsAsFactors = FALSE)
df = df[df$CDR3 != '',]
head(df)

df2 = read.csv('EoE_TCR_190322_PoolG_EoE_TR1V_TCRalignsortsortMappingSummary.txt', sep = '\t', header = 1, stringsAsFactors = FALSE)
df2 = df2[df2$CDR3 != '',]
head(df)

df = rbind(df, df2)

,UMI_Name,BC,UMI,BCNum,UMINum,nReads,TopVregion,TopJregion,TopJ_5UTR,CDR3,...,CDR3ReadCount,CDR3Freq,TRBV,TRBJ.5UTR,TRBJ,TRBC,TRAV,TRAJ.5UTR,TRAJ,TRAC
1,BC1UMI1,CAGACTCGTGGG,GGCAGCCA,1,1,613,TRBV7-2,TRBJ2-7,,CASSPRRDSYEQYF,...,535,0.98,612,0,612,0,8,0,23,0
2,BC2UMI1,TGCATGGATTAC,TAGAAAAA,2,1,604,TRBV20-1,TRBJ1-6,TRBJ2-6_5UTR,CSARDSGQGLNSPLHF,...,478,0.98,594,1,598,0,33,0,480,0
4,BC2UMI3,TGCATGGATTAC,CGGGGGGG,2,3,332,TRAV21,TRAJ37,TRAJ27_5UTR,CAPEGSSNTGKLIF,...,278,0.98,19,0,3,0,329,1,330,0
5,BC2UMI4,TGCATGGATTAC,TAGCTATG,2,4,190,TRAV26-1,TRAJ37,TRAJ53_5UTR,CIRGSSNTGKLIF,...,73,0.99,99,0,98,0,111,1,96,0
8,BC2UMI7,TGCATGGATTAC,GGTGGGGG,2,7,53,TRBV20-1,TRBJ1-6,,CSARDSGQGLNSPLHF,...,41,1.00,53,0,53,0,2,0,41,0
14,BC2UMI13,TGCATGGATTAC,GGGGACCG,2,13,28,TRBV12-4,TRBJ2-5,,CASSPTRQETQYF,...,1,0.09,28,0,28,0,11,0,0,0


,UMI_Name,BC,UMI,BCNum,UMINum,nReads,TopVregion,TopJregion,TopJ_5UTR,CDR3,...,CDR3ReadCount,CDR3Freq,TRBV,TRBJ.5UTR,TRBJ,TRBC,TRAV,TRAJ.5UTR,TRAJ,TRAC
1,BC1UMI1,CAGACTCGTGGG,GGCAGCCA,1,1,613,TRBV7-2,TRBJ2-7,,CASSPRRDSYEQYF,...,535,0.98,612,0,612,0,8,0,23,0
2,BC2UMI1,TGCATGGATTAC,TAGAAAAA,2,1,604,TRBV20-1,TRBJ1-6,TRBJ2-6_5UTR,CSARDSGQGLNSPLHF,...,478,0.98,594,1,598,0,33,0,480,0
4,BC2UMI3,TGCATGGATTAC,CGGGGGGG,2,3,332,TRAV21,TRAJ37,TRAJ27_5UTR,CAPEGSSNTGKLIF,...,278,0.98,19,0,3,0,329,1,330,0
5,BC2UMI4,TGCATGGATTAC,TAGCTATG,2,4,190,TRAV26-1,TRAJ37,TRAJ53_5UTR,CIRGSSNTGKLIF,...,73,0.99,99,0,98,0,111,1,96,0
8,BC2UMI7,TGCATGGATTAC,GGTGGGGG,2,7,53,TRBV20-1,TRBJ1-6,,CSARDSGQGLNSPLHF,...,41,1.00,53,0,53,0,2,0,41,0
14,BC2UMI13,TGCATGGATTAC,GGGGACCG,2,13,28,TRBV12-4,TRBJ2-5,,CASSPTRQETQYF,...,1,0.09,28,0,28,0,11,0,0,0


In [2]:
df$umi = paste0(df$BC, df$UMI)
df$id = rownames(df)
cdrs = unique(df$CDR3)
length(cdrs)

[1] 4650

In [3]:
library(dplyr)
library(stringdist)
i = 0
collisions = c()
df$BC = as.character(df$BC)
cdrs = unique(df$CDR3)
for (curr in cdrs) {
    if (i %% 1000 == 0) {
        message(i)
    }
    sub = df %>% filter(CDR3 == curr)
    umi = sub$UMI
    if (dim(sub)[1] > 1) {
        if (length(umi) > 1) {
            badumi = umi[duplicated(umi)]
            for (curr_bad in badumi) {
                relevant_bc = sub$BC[sub$UMI == curr_bad]
                curr_data = sub %>% filter(BC %in% relevant_bc)
                curr_data =  curr_data %>% group_by(BC) %>% summarize(nUMI = n(), nReads = sum(nReads)) %>% arrange(desc(nUMI), desc(nReads))
                bad = curr_data$BC[-1]
                bad = paste0(bad, curr_bad)
                bad
                collisions= c(collisions, bad)
            }
        }

    }
            i = i+ 1

}


Warning message:
"package 'dplyr' was built under R version 3.5.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'stringdist' was built under R version 3.5.3"0
1000
2000
3000
4000


In [4]:
keep = df %>% filter(!umi%in%collisions)
dim(keep)
write.table(keep, 'umi_collision.txt', col.names = TRUE, row.names = FALSE, sep= '\t')

[1] 17677    28